# Converting AHT to CRT using Session Level Data
## Terms
$T_n = Inbound Time$

$T_o = Outbound Time$

$N_n = NumberInboundCalls$

$N_o = NumberOutboundCalls$

## Metrics
## $AHT=\frac{T_n}{N_n}$

## $CRT=\frac{T_n+T_o}{N_n}$

## Therefore,
## $\mathbb{E}[CRT] = \mathbb{E}[AHT]+\mathbb{E}[T_o]*\frac{N_o}{N_n}$

Meaning the effect on $CRT$ is the effect on $AHT$ times the ratio of inbound calls to outbound calls.

In [1]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string.replace('$DATE$','2023-08-28')

def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def run_query(query):
    print("Query Start " + timestamp())
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    print("Query End " + timestamp())
    return df
    

def add_conditionals(df):
    df['Date']=pd.to_datetime(df['Date'])
    df['TestGroup'] = (df['Employee'].isin((
        '364717','426097','547655','552121',
        '554487','569375','572247','572815',
        '572909','573190','573192','573276',
        '573573','573585','575731','575996',
        '576565','576911','577073','577246',
        '579162','580895','580911','581015',
        '581139','581145','581275'
        ))).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    df['Weeks Ago'] = df['Week'].astype(int).max() - df['Week'].astype(int)
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    for queue in df['TaskQueue'].unique():
        df[queue] = (df['TaskQueue']==queue).astype(int)
    df['Constant'] =1
    return df

# Run Regression to Calc $\mathbb{E}[AHT]$ and $N_n$

In [11]:
df = run_query(file_to_string('AHT And Sales.sql'))
df = add_conditionals(df)
X_list = [
        'Weeks Ago',
        'ExpertAssistOnForThisUser',
        'Constant','TestGroup',
        'Offered','Accepted']
display(df[X_list].corr())
model = model = sm.OLS(
    endog=df['HandleTime'],
    exog=df[X_list]).fit()
display(model.summary())
Aht_Reduction = model.params['ExpertAssistOnForThisUser']
display(Aht_Reduction)
Number_Inbound = df['Constant'].sum()

# display(df.head(),df.describe())
df['Weeks Ago']=df['Weeks Ago']*-1


del df, model

Query Start 2023-10-31 03:01:20 PM


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_4076\3073750119.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


Query End 2023-10-31 03:01:33 PM


,Weeks Ago,ExpertAssistOnForThisUser,Constant,TestGroup,Offered,Accepted
Weeks Ago,1.00,-0.49,NaN,0.01,0.02,0.02
ExpertAssistOnForThisUser,-0.49,1.00,NaN,0.61,-0.03,-0.02
Constant,NaN,NaN,NaN,NaN,NaN,NaN
TestGroup,0.01,0.61,NaN,1.00,-0.03,-0.02
Offered,0.02,-0.03,NaN,-0.03,1.00,0.22
Accepted,0.02,-0.02,NaN,-0.02,0.22,1.00


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             HandleTime   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     272.9
Date:                Tue, 31 Oct 2023   Prob (F-statistic):          9.66e-283
Time:                        15:01:33   Log-Likelihood:            -1.5592e+05
No. Observations:               18941   AIC:                         3.119e+05
Df Residuals:                   18935   BIC:                         3.119e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Weeks Ago                   -16.6348      3.226     -5.156      0.000     -22.959     -10.311
ExpertAssistOnForThisUser   -50.3474     23.227     -2.168      0.030     -95.875      -4.820
Constant                    969.5250     20.086     48.268      0.000     930.154    1008.896
TestGroup                    54.6918     18.605      2.940      0.003      18.225      91.159
Offered                     279.3022     13.881     20.121      0.000     252.094     306.511
Accepted                    688.3012     26.890     25.597      0.000     635.595     741.007
==============================================================================
Omnibus:                     9294.186   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            80730.918
Skew:                           2.185   Prob(JB):                         0.00
Kurtosis:                      12.121   Cond. No.                         25.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

-50.347384446798834

# Run Regression to Calc $\mathbb{E}[T_o]$ and $N_o$

In [9]:
df = run_query(file_to_string('OutboundTime.SQL'))
df = add_conditionals(df)

model = sm.OLS(endog=df['ResolveTime'],exog=df[['Constant','ExpertAssistOnForThisUser','TestGroup']]).fit()
display(model.summary())
OutboundReduction = model.params['ExpertAssistOnForThisUser']
display(OutboundReduction)
Number_Outbound = df['Constant'].sum()

# display(df.head(),df.describe())

del df, model

Query Start 2023-10-31 03:00:07 PM


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_4076\3073750119.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


Query End 2023-10-31 03:00:10 PM


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            ResolveTime   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.964
Date:                Tue, 31 Oct 2023   Prob (F-statistic):           0.000950
Time:                        15:00:10   Log-Likelihood:                -80859.
No. Observations:               10439   AIC:                         1.617e+05
Df Residuals:                   10436   BIC:                         1.617e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Constant                    283.2879      7.530     37.619      0.000     268.527     298.049
ExpertAssistOnForThisUser   -41.3650     16.186     -2.556      0.011     -73.093      -9.637
TestGroup                    54.0041     14.484      3.729      0.000      25.613      82.395
==============================================================================
Omnibus:                     9235.213   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           398402.252
Skew:                           4.142   Prob(JB):                         0.00
Kurtosis:                      32.109   Cond. No.                         4.30
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

-41.36500043387083

# $\mathbb{E}[CRT] = \mathbb{E}[AHT]+\mathbb{E}[T_o]*\frac{N_o}{N_n}$

In [4]:
CRT_Reduction = Aht_Reduction+OutboundReduction*(Number_Outbound/Number_Inbound)
print(f"CRT={CRT_Reduction:.2f} = {Aht_Reduction:.2f}+{OutboundReduction:.2f}*{Number_Outbound/Number_Inbound:.2%}")
print(Number_Inbound)
print(Number_Outbound)

CRT=-73.14 = -50.35+-41.37*55.11%
18941
10439
